### Importing important libraries

In [201]:
import pandas as pd
import requests
import json
from your_passwort_file import airport_API, sql_root_passwort

#### Getting Citydata from SQL

In [203]:
schema = "gans"
host = "127.0.0.1"
user = "root"
password = sql_root_passwort
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
citydata_from_sql=pd.read_sql("city", con=connection_string)

In [179]:
citydata_from_sql

,city_id,city,longitude,latitude,country_id
0,1,Berlin,13.40500,52.5200,1
1,2,Paris,2.35222,48.8567,2
2,3,Hamburg,10.00000,53.5500,1


In [181]:
api_key=airport_API

In [153]:
import requests

url = "https://aerodatabox.p.rapidapi.com/airports/search/location"
querystring = {"lat":"52.311324","lon":"13.24","radiusKm":"50","limit":"2","withFlightInfoOnly":"true"}

#url = "https://aerodatabox.p.rapidapi.com/airports/search/location/52.31/13.24/km/50/16"
#querystring = {"withFlightInfoOnly":"true"}

headers = {
	"X-RapidAPI-Key": api_key,
	"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.json())

{'searchBy': {'lat': 52.311325, 'lon': 13.24}, 'count': 1, 'items': [{'icao': 'EDDB', 'iata': 'BER', 'name': 'Berlin Brandenburg', 'shortName': 'Brandenburg', 'municipalityName': 'Berlin', 'location': {'lat': 52.35139, 'lon': 13.493889}, 'countryCode': 'DE', 'timeZone': 'Europe/Berlin'}]}


In [113]:
response_list=response.json()
response_list['items']

[{'icao': 'EDDB',
  'iata': 'BER',
  'name': 'Berlin Brandenburg',
  'shortName': 'Brandenburg',
  'municipalityName': 'Berlin',
  'location': {'lat': 52.35139, 'lon': 13.493889},
  'countryCode': 'DE',
  'timeZone': 'Europe/Berlin'}]

In [115]:
response_list['items'][0].get('icao',None) #icao name for that airport

'EDDB'

In [117]:
response_list['items'][0].get('municipalityName',None)#name of the city

'Berlin'

In [119]:
response_list['items'][0].get('name',None)#airport name

'Berlin Brandenburg'

In [183]:
def get_airports(latitudes, longitudes):
  # API headers
    headers = {
    "X-RapidAPI-Key": api_key,
    "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
    }

    #querystring = {"withFlightInfoOnly": "true"}
    url = "https://aerodatabox.p.rapidapi.com/airports/search/location"
    
  # DataFrame to store results
    all_airports = []

    for lat, lon in zip(latitudes, longitudes):
        # Construct the URL with the latitude and longitude
        #url = f"https://aerodatabox.p.rapidapi.com/airports/search/location/{lat}/{lon}/km/50/16"
        querystring = {"lat":lat,"lon":lon,"radiusKm":"50","limit":"10","withFlightInfoOnly":"true"}

    # Make the API request
        response = requests.get(url, headers=headers, params=querystring)
        #print(response.text)

        if response.status_code == 200:
          data = response.json()
          #print(data)
          for item in data['items']:
              airports={'airport_icao':item.get('icao',None),
                         'airport_name':item.get('name',None),
                           'city':item.get('municipalityName',None)}
          #airports = pd.json_normalize(data.get('items', []))
              all_airports.append(airports)
            

    return pd.DataFrame(all_airports)

In [185]:
latitudes =citydata_from_sql['latitude']
longitudes =citydata_from_sql['longitude']
    
city_airport_df=get_airports(latitudes, longitudes)

In [187]:
city_airport_df

,airport_icao,airport_name,city
0,EDDT,Berlin -Tegel,Berlin
1,EDDB,Berlin Brandenburg,Berlin
2,LFPB,Paris -Le Bourget,Paris
3,LFPO,Paris -Orly,Paris
4,LFPG,Paris Charles de Gaulle,Paris
5,EDDH,Hamburg,Hamburg


In [189]:
city_airport= city_airport_df.merge(citydata_from_sql,
                                   on = "city",
                                   how="left")


In [191]:
city_airport

,airport_icao,airport_name,city,city_id,longitude,latitude,country_id
0,EDDT,Berlin -Tegel,Berlin,1,13.40500,52.5200,1
1,EDDB,Berlin Brandenburg,Berlin,1,13.40500,52.5200,1
2,LFPB,Paris -Le Bourget,Paris,2,2.35222,48.8567,2
3,LFPO,Paris -Orly,Paris,2,2.35222,48.8567,2
4,LFPG,Paris Charles de Gaulle,Paris,2,2.35222,48.8567,2
5,EDDH,Hamburg,Hamburg,3,10.00000,53.5500,1


In [193]:
city_airport.drop(columns=["city","longitude","latitude","country_id"],inplace=True)


In [195]:
city_airport

,airport_icao,airport_name,city_id
0,EDDT,Berlin -Tegel,1
1,EDDB,Berlin Brandenburg,1
2,LFPB,Paris -Le Bourget,2
3,LFPO,Paris -Orly,2
4,LFPG,Paris Charles de Gaulle,2
5,EDDH,Hamburg,3


In [215]:
city_airport.to_sql('city_airport',
                  if_exists='append',
                  con=connection_string,
                  index=False)

6

In [217]:
city_airport_from_sql=pd.read_sql('city_airport',
                                 con=connection_string)

In [219]:
city_airport_from_sql

,icao_id,airport_icao,airport_name,city_id
0,1,EDDT,Berlin -Tegel,1
1,2,EDDB,Berlin Brandenburg,1
2,3,LFPB,Paris -Le Bourget,2
3,4,LFPO,Paris -Orly,2
4,5,LFPG,Paris Charles de Gaulle,2
5,6,EDDH,Hamburg,3


### Creating function with flight details

In [289]:
import pandas as pd
from datetime import datetime, timedelta
import requests
from pytz import timezone

def flight(icaos):
    
    api_key=airport_API
    berlin_timezone = timezone('Europe/Berlin')
    today = datetime.now(berlin_timezone).date()
    tomorrow = (today + timedelta(days=1))

    
    flight_details=[]

#icao="EGLC"
    for icao in icaos: 

        # the api can only make 12 hour calls, therefore, two 12 hour calls make a full day
        # using the nested lists below we can make a morning call and extract the data
        # then make an afternoon call and extract the data
        times = [["00:00","11:59"],
                ["12:00","23:59"]]

        for time in times:

            url = (f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}")

            querystring = {"withLeg":"true","direction":"Both","withCancelled":"false","withCodeshared":"true","withCargo":"false","withPrivate":"false","withLocation":"false"}

            headers = {
            	"x-rapidapi-key": api_key,
            	"x-rapidapi-host": "aerodatabox.p.rapidapi.com"
                    }

            response = requests.get(url, headers=headers, params=querystring)
            if response.status_code == 200:
                print('done')
                response_list=response.json()
                retrieval_time = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")
        
                for item in response_list['arrivals']:
                    flight_detail={
                        "flight_num":item.get("number",None),
                        "departure_icao":item['departure']['airport'].get('icao',None),
                        "arrival_icao":icao,
                        "scheduled_arrival_time": item.get("departure", {}).get("scheduledTime", {}).get("local", None),
                        "data_retrieved_at":retrieval_time
                    }
                    
                flight_details.append(flight_detail)
    flights_df = pd.DataFrame(flight_details)
    flights_df["scheduled_arrival_time"] = flights_df["scheduled_arrival_time"].str[:-6]
    flights_df["scheduled_arrival_time"] = pd.to_datetime(flights_df["scheduled_arrival_time"])
    flights_df["data_retrieved_at"] = pd.to_datetime(flights_df["data_retrieved_at"])
        
    return flights_df
    

In [299]:
icao_list =city_airport_from_sql['airport_icao']
flights_detail=flight(icao_list)
#get_flight_data(icao_list)

done
done
done
done
done
done
done
done


In [301]:
flights_detail

,flight_num,departure_icao,arrival_icao,scheduled_arrival_time,data_retrieved_at
0,LH 180,EDDF,EDDB,2025-03-21 10:45:00,2025-03-20 17:12:58
1,I2 1809,LEMD,EDDB,2025-03-21 20:05:00,2025-03-20 17:12:58
2,TO 3911,LIRF,LFPO,2025-03-21 09:40:00,2025-03-20 17:12:59
3,TP 442,LPPT,LFPO,2025-03-21 19:30:00,2025-03-20 17:12:59
4,AF 459,SBGR,LFPG,2025-03-20 20:30:00,2025-03-20 17:12:59
5,IB 591,LEMD,LFPG,2025-03-21 21:40:00,2025-03-20 17:13:00
6,SK 1647,EKCH,EDDH,2025-03-21 10:55:00,2025-03-20 17:13:00
7,DE 1519,GCLP,EDDH,2025-03-21 16:40:00,2025-03-20 17:13:00


In [304]:
flights=flights_detail.merge(city_airport_from_sql,
                             left_on='arrival_icao',right_on='airport_icao',
                             how='left'
                            )

In [310]:
flights.drop(columns=['airport_icao','airport_name','city_id'],inplace=True)

In [312]:
flights

,flight_num,departure_icao,arrival_icao,scheduled_arrival_time,data_retrieved_at,icao_id
0,LH 180,EDDF,EDDB,2025-03-21 10:45:00,2025-03-20 17:12:58,2
1,I2 1809,LEMD,EDDB,2025-03-21 20:05:00,2025-03-20 17:12:58,2
2,TO 3911,LIRF,LFPO,2025-03-21 09:40:00,2025-03-20 17:12:59,4
3,TP 442,LPPT,LFPO,2025-03-21 19:30:00,2025-03-20 17:12:59,4
4,AF 459,SBGR,LFPG,2025-03-20 20:30:00,2025-03-20 17:12:59,5
5,IB 591,LEMD,LFPG,2025-03-21 21:40:00,2025-03-20 17:13:00,5
6,SK 1647,EKCH,EDDH,2025-03-21 10:55:00,2025-03-20 17:13:00,6
7,DE 1519,GCLP,EDDH,2025-03-21 16:40:00,2025-03-20 17:13:00,6


### Connecting Flight table into SQL

In [315]:
flights.to_sql('flights',
              if_exists='append',
                  con=connection_string,
                  index=False)

8